In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

In [2]:
#export

from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s


In [3]:
(x_train, y_train, x_valid, y_valid) = get_data() 

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
#normalize x_valid with x_train mean and std
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(-7.6999e-06), tensor(1.))

In [7]:
#export
def test_near_zero(a, tol=1e-3): assert a.abs() < tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(x_train.std()-1)

In [9]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Basic architecture

In [10]:
nh = 50  # numbder of hidden neurons

In [11]:
w1 = torch.randn(m, nh) / math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1) / math.sqrt(nh)
b2 = torch.zeros(1)

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [13]:
w1.std()-1

tensor(-0.9642)

In [14]:
x_valid.mean(), x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [15]:
def lin(x, w, b): return x@w + b

In [16]:
x_valid.shape, w1.shape, b1.shape


(torch.Size([10000, 784]), torch.Size([784, 50]), torch.Size([50]))

In [17]:
t = lin(x_valid, w1 ,b1)

In [18]:
t.mean(), t.std()

(tensor(-0.0047), tensor(0.9736))

In [19]:
def relu(x): return x.clamp_min(min=0)

In [20]:
t = relu(lin(x_valid, w1, b1))

In [21]:
t.mean(), t.std()

(tensor(0.3807), tensor(0.5707))

# Kaiming init

In [22]:
w1 = torch.randn(m, nh) * math.sqrt(2/m)  # Only change to 2/m
b1 = torch.zeros(nh)

In [23]:
w1.mean(), w1.std()

(tensor(-0.0005), tensor(0.0506))

In [24]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()   # to fix  the std, very close to 1 here

(tensor(0.5764), tensor(0.8430))

In [25]:
#export
from torch.nn import init

In [26]:
# This is how it's supposed to be
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [27]:
w1.mean(),w1.std()

(tensor(2.3604e-05), tensor(0.0509))

In [28]:
t.mean(), t.std()

(tensor(0.6215), tensor(0.8634))

In [29]:
w1.shape, t.shape

(torch.Size([784, 50]), torch.Size([10000, 50]))

In [30]:
import torch.nn

In [31]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [32]:
# redefine relu
def relu(x): return x.clamp_min(0.) - 0.5

In [33]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.1389), tensor(0.8770))

In [34]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [35]:
%timeit -n 10 _ = model(x_valid)

5.12 ms ± 241 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

## Loss Function: MSE

In [37]:
model(x_valid).shape

torch.Size([10000, 1])

In [38]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [74]:
y_train, y_valid = y_train.float(), y_valid.float()

In [ ]:
preds = model(x_train)
preds.shape

In [ ]:
%time mse(preds, y_train)

## Backward gradient

In [ ]:
# last layer, mse backward
def mse_grad(inp, targ):
    inp.g = 2 * ( inp.squeeze()  - targ).unsqueeze(-1) / inp.shape[0]

In [ ]:
def relu_grad(inp, out):
    # with respect to the input
    inp.g = (inp>0).float() * out.g

In [ ]:
def lin_grad(inp, out, w, b):
    # out = input@w + b
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0) # WHY SUM? 
    b.g = out.g.sum(0)

In [ ]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [ ]:
forward_and_backward(x_train, y_train)

## testing later

In [ ]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [ ]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [ ]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [ ]:
loss = forward(xt2, y_train)

In [ ]:
loss.backward()

In [ ]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

# Layers as classes

In [61]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [62]:
class Lin():
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = self.inp @ self.w + self.b
        return self.out
        
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [68]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

Type them up

In [69]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse() 
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()   # store in each of component
        

In [70]:
w1.g,b1.g,w2.g,b2.g = [None]*4   # zero out the gradient
model = Model(w1, b1, w2, b2)

In [71]:
x_train.shape, y_train.shape

(torch.Size([50000, 784]), torch.Size([50000]))

In [73]:
y_train[0]

tensor(5)

In [75]:
%time loss = model(x_train, y_train)

CPU times: user 115 ms, sys: 0 ns, total: 115 ms
Wall time: 33 ms


In [76]:
%time model.backward()

CPU times: user 4.41 s, sys: 1.73 s, total: 6.15 s
Wall time: 1.54 s


In [77]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

NameError: name 'w2g' is not defined

# More refactering

In [89]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [90]:
# redefine
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [91]:
class Lin(Module):
    
    def __init__(self, w, b):
        self.w, self.b = w, b
    
    def forward(self, inp ):
        return inp @ self.w + self.b
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [92]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [93]:
# Putting things together
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse() 
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()   # store in each of component
        
    

In [94]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [95]:
%time loss = model(x_train, y_train)

CPU times: user 109 ms, sys: 45 µs, total: 109 ms
Wall time: 29.1 ms


In [96]:
%time model.backward()

CPU times: user 327 ms, sys: 47.6 ms, total: 375 ms
Wall time: 96.8 ms


# Use nn.linear and nn.Module

In [97]:
from torch import nn

In [104]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [105]:
model = Model(m, nh, 1)

In [106]:
%time loss = model(x_train, y_train)

CPU times: user 109 ms, sys: 8.58 ms, total: 118 ms
Wall time: 29.6 ms


In [107]:
%time loss.backward()

CPU times: user 329 ms, sys: 32 ms, total: 361 ms
Wall time: 90.9 ms
